In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install transformers[torch] datasets torch peft trl==0.4.7 bitsandbytes accelerate cuda-python

In [ ]:
# !zip -r ./fine_tuned_model.zip ./fine_tuned_model
# pass

In [ ]:
# !unzip ./lora_model.zip
# pass

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
# Init imports and constants

import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, AutoTokenizer, pipeline
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig, PeftModel, get_peft_model

BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
dataset = load_dataset("dair-ai/emotion", split="train")

In [ ]:
# Loading model and tokenizer

quant_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    quantization_config=quant_config,
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

def merge_columns(example):
  emotion_mappings = {
      0: "sadness",
      1: "joy",
      2: "love",
      3: "anger",
      4: "fear",
      5: "surprise"
  }

  merged = example["text"] + " ->: " + emotion_mappings[example["label"]]

  messages = [
      {
          "role": "user",
          "content": example["text"] + " ->: "
      },
      {
          "role": "assistant",
          "content": merged
      }
  ]

  example["prediction"] = messages

  return example

predictions = dataset.map(merge_columns)
formatted_dataset = predictions.map(lambda row: {"formatted": tokenizer.apply_chat_template(row["prediction"], tokenize=False, add_generation_prompt=False)})

In [ ]:
print(base_model.get_memory_footprint() / 1**9)

In [ ]:
lora_config = LoraConfig(
    r=4,
    lora_alpha=8,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

lora_model = get_peft_model(base_model, lora_config)

In [ ]:
training_arguments = TrainingArguments(
    output_dir="training_output",
    num_train_epochs=1,
    logging_steps=100,
    save_steps=100,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    warmup_steps=100,
)

trainer = SFTTrainer(
    model=lora_model,
    train_dataset=formatted_dataset,
    dataset_text_field="formatted",
    args=training_arguments
)

trainer.train()
trainer.save_model("./lora_model")

In [ ]:
full_base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    trust_remote_code=True,
)

lora_adapter = PeftModel.from_pretrained(
    full_base_model,
    "./lora_model"
)

fine_tuned_model = lora_adapter.merge_and_unload()

fine_tuned_model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

In [ ]:
# Testing original model

text_gen = pipeline(
    "text-generation",
    model=BASE_MODEL,
)

messages = [
    {
        "role": "user",
        "content": "i am ever feeling nostalgic about the fireplace i will know that it is still on the property ->: ",
    },
]

prompt = text_gen.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
result = text_gen(prompt)
print(result[0]["generated_text"])

In [ ]:
# Testing fine tuned model

fine_text_gen = pipeline(
    "text-generation",
    model=fine_tuned_model,
    tokenizer=tokenizer
)

messages = [
    {
        "role": "user",
        "content": "i am ever feeling nostalgic about the fireplace i will know that it is still on the property ->: ",
    },
]

prompt = fine_text_gen.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
result = fine_text_gen(prompt)
print(result[0]["generated_text"])